In [16]:
from __future__ import division
import pandas as pd
import numpy as np
from parth import *
import pickle

from multiprocessing import cpu_count
import concurrent.futures

print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')

pandas version: 1.3.5
numpy version: 1.20.3


In [17]:
# Some input values (i.e. DNNU) modify a diffrent output parameter (N_nu)
parth_inout_map = {
    'DNNU': 'N_nu',
    'ETA10': 'eta10',
    'TAU': 'tau'
}

# !!! will need to update card_mod if you change obs
# k, v of intrested obs: expermental errors 
observables = {
    'H2/H': 0.03* 10**(-5),     # https://arxiv.org/abs/1710.11129       
    'Y_p': 0.0026,     # https://arxiv.org/abs/1904.01594 8 26 22
    'tau': 0.5          #pdg       
}

# See https://arxiv.org/pdf/0705.0290.pdf Table I
# Modifcations to the base card to always be considered
card_mod = {
    'OUTPUT': 'F  2  3 6'
}

# params to vary over and step size, uses parth's card syntax
parameters = {
    'ETA10': 0.005,
    'DNNU': 0.036, 
    'TAU': 0.5
}

# cleans up output
obs_and_params = list(observables.keys()) + [parth_inout_map.get(k, k) for k in parameters.keys()]

# otherwise we will just find fisher
use_dali = True

save = True
save_file = 'bbn__small_t' + str(observables['tau']) + '.pkl'

# these are to match lesnpower output so we can just use the cobaya dali program
lo_spec_id = 'unlensed'
lo_experment = 0 

In [18]:
def dp(val):
    display(val)

def _run(param, mod):
    nope = Pyrthenope(card_mod=mod)
    for (k, v) in param:
        nope.card[k] = nope.card[k] + v
    return nope.run()    

def runner(obs, params, cardMods=None, doDali=True, numThreads=cpu_count()):
    # This generates all of our runs, these will run automagically and will only halt when a corresponding .result() is called
    processes = np.empty((2**doDali, len(params), len(params), 2**(1+doDali)), dtype=object)
    with concurrent.futures.ThreadPoolExecutor(max_workers=numThreads) as exec:
        for i, (param1, step1) in enumerate(params.items()):
            processes[0, i, 0, 0] = exec.submit(_run, [(param1, -step1/2)], cardMods)
            processes[0, i, 0, 1] = exec.submit(_run, [(param1,  step1/2)], cardMods)
            if doDali:
                for j, (param2, step2) in enumerate(params.items()):
                    processes[1, i, j, 0] = exec.submit(_run, [(param1, -step1/2), (param2, -step2/2)], cardMods)
                    processes[1, i, j, 1] = exec.submit(_run, [(param1,  step1/2), (param2, -step2/2)], cardMods)
                    processes[1, i, j, 2] = exec.submit(_run, [(param1, -step1/2), (param2,  step2/2)], cardMods)
                    processes[1, i, j, 3] = exec.submit(_run, [(param1,  step1/2), (param2,  step2/2)], cardMods)

    # derivative vectors
    d1Vec = pd.DataFrame()
    d2Vec = pd.DataFrame()

    #calculate the derivatives
    for i, (param1, step1) in enumerate(params.items()):
        v01 = processes[0, i, 0, 0].result()
        v02 = processes[0, i, 0, 1].result()
        
        d1 = pd.DataFrame((v02[obs] - v01[obs]) / step1).rename(index={0:param1})    
        d1Vec = pd.concat([d1Vec, d1])

        if doDali:
            for j, (param2, step2) in enumerate(params.items()):
                v11 = processes[1, i, j, 0].result()
                v12 = processes[1, i, j, 1].result()
                v13 = processes[1, i, j, 2].result()
                v14 = processes[1, i, j, 3].result()

                d11 = (v12[obs] - v11[obs]) / step1
                d12 = (v14[obs] - v13[obs]) / step1
                d2 = pd.DataFrame((d12 - d11) / step2).rename(index={0:(param1, param2)})
                d2Vec = pd.concat([d2Vec, d2])

    # Finds the (gaussian) 1/sigma^2 errors
    errors = np.array([list(obs.values())])
    isigma2 = pd.DataFrame(
        np.linalg.inv(errors.T * np.identity(len(list(obs))) * errors), 
        index=[list(obs.keys())], columns=[list(obs.keys())])

    # calculate fisher, dali3 and dali4
    # https://arxiv.org/pdf/1401.6892.pdf 15
    fisher = np.einsum('ia,ab,jb', d1Vec, isigma2, d1Vec)
    if doDali:
        # generate tensors so we can use these in np.einsum
        d2Ten = d2Vec.values.reshape(len(params), len(params), len(obs))

        dali3 = np.einsum('ija,ab,kb', d2Ten, isigma2, d1Vec)
        dali4 = np.einsum('ija,ab,klb', d2Ten, isigma2, d2Ten)

        return fisher, dali3, dali4, isigma2

    return fisher, None, None, isigma2


In [19]:
baseData = _run([], card_mod)
# dp(baseData)

fisher, dali3, dali4, is2 = runner(observables, parameters, card_mod, doDali=use_dali)

In [ ]:
if save:
    fid_values = {
        'omega_b_h2': baseData['OmegaBh^2'].values[0], #0.02242 # check that this corresponds to our fid eta10 6.13332, is this needed?
        'N_nu': baseData['N_nu'].values[0], #3.0,
        'tau': baseData['tau'].values[0]
    }
    saveData = {'cosmoFid': fid_values, 'fisherGaussian': {lo_experment: {lo_spec_id: fisher}}, 'iSigma2': is2}
    if use_dali:
        saveData['DALI3Gaussian'] =  {lo_experment: {lo_spec_id: dali3}}
        saveData['DALI4Gaussian'] =  {lo_experment: {lo_spec_id: dali4}}

    with open(save_file, 'wb') as file:
        pickle.dump(saveData, file, protocol=pickle.HIGHEST_PROTOCOL)
        
    